# STEP 1


*   刪除每月淨利若有負的股票
*   刪除本益比極端股票(本益比平均大於所有股票本益比平均的1.5倍)



In [1]:
pip install jmespath

In [2]:
import requests
import json
import jmespath
import pandas as pd
import time
import re
from lxml import etree
from bs4 import BeautifulSoup

In [ ]:
stocklist=["1201","1203","1210","1213","1215","1216","1217","1218","1219","1220","1225","1227","1229","1231","1232","1233","1234","1235","1236","1256","1702","1737"]

In [ ]:
datelist=[]
for year in range(2019,2022):
    for month in range(1,13):
        date=str(year)+str("%02d" % month)+"01"
        if date == '20210401':
          break
        else:
          datelist.append(date)
    
#食品股
stocklist=["1201","1203","1210","1213","1215","1216","1217","1218","1219","1220","1225","1227","1229","1231","1232","1233","1234","1235","1236","1256","1702","1737"]
data={"STOCK":[],"年分":[],"本益比":[]}

for date in datelist:
  try:
    for stock in stocklist:
      request_url = f'https://www.twse.com.tw/exchangeReport/BWIBBU?response=json&date={date}&stockNo={stock}'
      res = requests.get(request_url)
      resinfo = json.loads(res.text)
      peratio=jmespath.search('data[:][3][]',resinfo)
      for peratioidx in range(len(peratio)):
        try:
          data['本益比'].append(float(peratio[peratioidx]))
          data['STOCK'].append(stock)
          data['年分'].append((re.findall(r'^2\w{3}', date)[0]))
        except TypeError:
          print(peratioidx)
          pass
      time.sleep(3)
  except ValueError:
    stocklist.remove(stock)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
dfmax = df.groupby(['STOCK','年分']).max()
dfmin = df.groupby(['STOCK','年分']).min()
dfmean = df.groupby(['STOCK','年分']).mean()

In [ ]:
new = pd.concat([dfmean, dfmin, dfmax], axis=1)
new.columns=['平均本益比','最小本益比','最大本益比']

In [ ]:
new

平均本益比   最小本益比   最大本益比
STOCK 年分                              
1201  2019    8.282025    6.08   14.48
      2020   26.831673    6.37   45.59
      2021   27.750727   20.33   29.73
1203  2019   13.866033   12.96   14.84
      2020   16.171959   13.65   19.74
      2021   18.756000   17.90   21.10
1210  2019   14.294215   10.94   18.18
      2020   13.936571   11.03   15.71
      2021   14.266909   13.46   15.45
1215  2019   15.989005   13.30   21.33
      2020   12.234286   10.18   14.15
      2021   11.214364   10.68   12.39
1216  2019   21.735068   10.24   25.62
      2020   20.654041   17.76   23.31
      2021   18.632545   17.57   19.26
1217  2019   74.610588   17.48  150.40
      2020   46.414898   19.00   72.40
      2021   21.536182   19.17   24.76
1218  2019   20.063937   18.91   21.93
      2020   17.775102   14.56   24.06
      2021   17.757818   16.49   18.89
1219  2019  108.455938  107.06  109.41
1220  2019   24.344627   18.04   28.85
      2020   14.755061   12.47   19.02
      2021   12.836727   11.98   14.27
1225  2019   49.184726   22.05   89.51
      2020   22.616816   19.60   26.42
      2021   25.323273   21.96   31.40
1227  2019   17.599154   15.63   20.45
      2020   17.211633   14.83   20.37
      2021   14.866000   14.19   16.55
1229  2019   13.647313   12.16   15.02
      2020   18.614816   11.27   26.85
      2021   20.119273   19.10   22.04
1231  2019   14.001642   12.30   15.32
      2020   13.745102   10.73   15.46
      2021   13.263273   12.58   15.80
1232  2019   19.176617   16.77   20.83
      2020   19.209265   14.50   21.53
      2021   19.095818   17.08   20.38
1233  2019   20.464876   18.81   23.02
      2020   31.413143   17.84   47.70
      2021   47.535818   44.80   55.87
1234  2019   17.410299   15.82   18.80
      2020   15.916612   13.83   17.41
      2021   15.013273   14.66   15.66
1235  2019  113.972090   10.15  487.50
      2020  112.548531   26.13  406.67
      2021   34.155455   31.15   44.85
1256  2019   12.882713   11.45   15.80
      2020   14.875918   10.37   19.93
      2021   16.422000   13.75   19.97
1702  2019   13.275532   12.54   14.18
      2020   12.134694    9.02   13.93
      2021   11.666909   11.07   12.42
1737  2019   17.081809   13.87   20.21
      2020   19.581306   15.77   21.59
      2021   20.285455   18.31   20.79

In [ ]:
new.to_csv('stockstep1.csv')

In [7]:
df1 = pd.read_csv('stockstep1.csv')

In [8]:
df1

,STOCK,年分,平均本益比,最小本益比,最大本益比
0,1201,2019,8.282025,6.08,14.48
1,1201,2020,26.831673,6.37,45.59
2,1201,2021,27.750727,20.33,29.73
3,1203,2019,13.866033,12.96,14.84
4,1203,2020,16.171959,13.65,19.74
5,1203,2021,18.756000,17.90,21.10
6,1210,2019,14.294215,10.94,18.18
7,1210,2020,13.936571,11.03,15.71
8,1210,2021,14.266909,13.46,15.45
9,1215,2019,15.989005,13.30,21.33


# STEP 2

*   算出合理區間
*   Link：https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1201&SYEAR=2020&SSEASON=4&REPORT_ID=C





### 找出PERatio

In [9]:
### ERROR??? ###
# from lxml import etree
# link = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1201&SYEAR=2020&SSEASON=4&REPORT_ID=C'
# res = requests.get(link)
# raw_data = etree.HTML(res.content)
# result = raw_data.xpath("//span[contains(text(),'基本每股盈餘合計')]/parent::td//following-sibling::td//text()")
# print(len(result))

In [10]:
# for stockidx in stocklist:
#   res = requests.get(f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={stockidx}&SYEAR=2020&SSEASON=4&REPORT_ID=C')
#   soup = BeautifulSoup(res.content,'lxml')
#   peratio = soup.find("ix:nonfraction", name_= "ifrs-full:BasicEarningsLossPerShare"))

In [11]:
#BY：https://www.finlab.tw/python-%E8%B2%A1%E5%A0%B1%E7%88%AC%E8%9F%B2-1-%E7%B6%9C%E5%90%88%E6%90%8D%E7%9B%8A%E8%A1%A8/

# import requests
# import pandas as pd
# import numpy as np

# def financial_statement(year, season, type='綜合損益彙總表'):

#     if year >= 1000:
#         year -= 1911

#     if type == '綜合損益彙總表':
#         url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
#     elif type == '資產負債彙總表':
#         url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
#     elif type == '營益分析彙總表':
#         url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
#     else:
#         print('type does not match')

#     r = requests.post(url, {
#         'encodeURIComponent':1,
#         'step':1,
#         'firstin':1,
#         'off':1,
#         'TYPEK':'sii',
#         'year':str(year),
#         'season':str(season),
#     })

#     r.encoding = 'utf8'
#     dfs = pd.read_html(r.text, header=None)

#     return pd.concat(dfs[1:], axis=0, sort=False)\
#              .set_index(['公司代號'])\
#              .apply(lambda s: pd.to_numeric(s, errors='ceorce'))

# financial_statement(2019,1,type='綜合損益彙總表')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir('/content/drive/My Drive/股票CSV') 

In [6]:
os.listdir()

['stockstep1.csv',
 'stock_1.csv',
 'stock_2.csv',
 'stock_3.csv',
 'stock_4.csv',
 'stock_6.csv',
 'stock_7.csv',
 'stock_8.csv',
 'stock_9.csv',
 'stock_5.csv',
 'stockstep2.csv',
 'stockstep2_1.csv',
 'stockstep2_2.csv']

In [12]:
# # 合併6個檔案
# read = pd.read_csv('108_1.csv', encoding='big5', low_memory=False)
# read.to_csv('stockAll.csv',encoding="big5",index=False)
# for i in range(2, 7):
#     read = pd.read_csv('AG_PROFILE_201912_FIN_{}.csv'.format(i), encoding='big5', low_memory=False)
#     read.to_csv('AG_PROFILE_201912_FIN.csv',encoding="big5",index=False, header=False, mode='a+')

# 合併9個檔案
read = pd.read_csv('stock_4.csv')
read.to_csv('stockstep2.csv')
for i in range(8, 10):
    read = pd.read_csv('stock_{}.csv'.format(i))
    read.to_csv('stockstep2.csv',header=False, mode='a+')

In [34]:
df2 = pd.read_csv('stockstep2.csv', index_col='Unnamed: 0')

In [39]:
stocklist = [str(i) for i in list(set(df1['STOCK']))]
df2 = df2[df2['公司代號'].isin(stocklist)]

In [40]:
dfmean = df2.groupby(['公司代號']).sum()/2.25
dfmean = dfmean.drop(dfmean.columns[:-1], axis=1)

In [41]:
dfmean

,基本每股盈餘（元）
公司代號,
1201,1.733333
1203,1.902222
1210,3.524444
1215,5.817778
1216,3.595556
1217,0.315556
1218,1.462222
1219,1.453333
1220,0.933333


In [42]:
dfmean.to_csv('stockstep2_1.csv')

### 算出合理區間

In [43]:
df2 = pd.read_csv('stockstep2_1.csv')
df2 = df2.rename(columns={'公司代號': 'STOCK','基本每股盈餘（元）': 'EPS'})

In [44]:
df2

,STOCK,EPS
0,1201,1.733333
1,1203,1.902222
2,1210,3.524444
3,1215,5.817778
4,1216,3.595556
5,1217,0.315556
6,1218,1.462222
7,1219,1.453333
8,1220,0.933333
9,1225,1.857778


In [45]:
df = pd.merge(df1,df2,on='STOCK')

In [46]:
df

,STOCK,年分,平均本益比,最小本益比,最大本益比,EPS
0,1201,2019,8.282025,6.08,14.48,1.733333
1,1201,2020,26.831673,6.37,45.59,1.733333
2,1201,2021,27.750727,20.33,29.73,1.733333
3,1203,2019,13.866033,12.96,14.84,1.902222
4,1203,2020,16.171959,13.65,19.74,1.902222
5,1203,2021,18.756000,17.90,21.10,1.902222
6,1210,2019,14.294215,10.94,18.18,3.524444
7,1210,2020,13.936571,11.03,15.71,3.524444
8,1210,2021,14.266909,13.46,15.45,3.524444
9,1215,2019,15.989005,13.30,21.33,5.817778


In [47]:
dfmax = df.groupby(['STOCK']).max().filter(['STOCK','最大本益比'], axis=1)
dfmin = df.groupby(['STOCK']).min().filter(['STOCK','最小本益比'], axis=1)
dfmean = df.groupby(['STOCK']).mean().filter(['STOCK','平均本益比','EPS'], axis=1)

In [48]:
df = pd.concat([dfmin, dfmax, dfmean], axis=1)

In [49]:
df

,最小本益比,最大本益比,平均本益比,EPS
STOCK,,,,
1201,6.08,45.59,20.954809,1.733333
1203,12.96,21.10,16.264664,1.902222
1210,10.94,18.18,14.165898,3.524444
1215,10.18,21.33,13.145885,5.817778
1216,10.24,25.62,20.340551,3.595556
1217,17.48,150.40,47.520556,0.315556
1218,14.56,24.06,18.532286,1.462222
1219,107.06,109.41,108.455938,1.453333
1220,11.98,28.85,17.312138,0.933333


In [50]:
df['便宜股價'] = df['最小本益比']*df['EPS']
df['安全股價'] = ((df['最小本益比']+df['平均本益比'])/2)*df['EPS']
df['合理股價'] = df['平均本益比']*df['EPS']
df['觀察股價'] = ((df['最大本益比']+df['平均本益比'])/2)*df['EPS']
df['昂貴股價'] = df['最大本益比']*df['EPS']

In [53]:
df = df.round(5)

In [54]:
df

,最小本益比,最大本益比,平均本益比,EPS,便宜股價,安全股價,合理股價,觀察股價,昂貴股價
STOCK,,,,,,,,,
1201,6.08,45.59,20.95481,1.73333,10.53867,23.43017,36.32167,57.67217,79.02267
1203,12.96,21.10,16.26466,1.90222,24.65280,27.79590,30.93901,35.53795,40.13689
1210,10.94,18.18,14.16590,3.52444,38.55742,44.24217,49.92692,57.00066,64.07440
1215,10.18,21.33,13.14588,5.81778,59.22498,67.85241,76.47984,100.28652,124.09320
1216,10.24,25.62,20.34055,3.59556,36.81849,54.97704,73.13558,82.62686,92.11813
1217,17.48,150.40,47.52056,0.31556,5.51591,10.25564,14.99538,31.22747,47.45956
1218,14.56,24.06,18.53229,1.46222,21.28996,24.19414,27.09832,31.13969,35.18107
1219,107.06,109.41,108.45594,1.45333,155.59387,156.60825,157.62263,158.31591,159.00920
1220,11.98,28.85,17.31214,0.93333,11.18133,13.66966,16.15800,21.54233,26.92667


In [55]:
df.to_csv('stockstep2_2.csv')